# **OPTIMIZATION OF VARIABLE STIFFNESS CYLINDRICAL SHELL**

In this notebook the optimized neural network will be used for the optimization of the fibers path parameters in order to achieve the maximum buckling load. To this end a bio-inspired metaheuristic algorithm, named PSO, will be used.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from copy import copy
import random
import time
import sys
import os
import pyDOE
from scipy.optimize import fsolve

In [2]:
# PyTorch
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

# Matplotlib spec
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']}) # Palatino font
plt.rcParams['pdf.fonttype'] = 42

For reproducibility:

In [3]:
seed = 0
np.random.seed(seed=seed)
torch.manual_seed(seed=seed)
random.seed(seed)

We need to import the normalization variables in order to normalize the particles position and recover the output of the networks. These normalization variables must be the same used during the training of the network.

In [4]:
# Modify parameter to choose the output folder to consider
load_case = 'torsion'
stacking_sequence = 'symmetric_balanced'
data_set = '8x'
fiber_path = 'harmlin'

input_folder = '../metamodel/' + load_case + '/' + stacking_sequence + '/' + data_set + '/' + fiber_path + '/'

x_norm = pd.read_csv(input_folder + 'X_bounds.csv', sep=",", index_col=0)
y_norm = pd.read_csv(input_folder + 'Y_bounds.csv', sep=",", index_col=0)

Now we have to import some function previously defined in the metamodel notebook.

In [5]:
def range_norm(x, x_min=None, x_max=None):
    """ Normalization in range [0, 1] """
    if x_min is None and x_max is None:
        x_min = np.min(x, axis=0)
        x_max = np.max(x, axis=0)
    x_norm = (x - x_min) / (x_max - x_min)

    return x_norm, x_min, x_max

def std_norm(x, m=None, s=None):
    """ Normalization with zero mean and unitary standard deviation """
    if m is None and s is None:
        m = np.mean(x, axis=0)
        s = np.std(x, axis=0)
    x_norm = (x - m) / s
    
    return x_norm, m, s

def reverse_range_norm(x_norm, x_min, x_max):
    """ Inverse of normalization in range [0, 1] """
    x = x_min +  x_norm * (x_max - x_min)

    return x

We need to know how many free plies we have

In [6]:
info = pd.read_csv(input_folder + 'model_info.csv', sep=",")
info.index = ['Value']
eff_plies = int(info['EffectivePlies'].values)
h = info['Height'].values
mesh_size = info['MeshSize'].values
k_max = info['MaxCurvature'].values
info

,Height,Radius,MaxCurvature,MeshSize,Plies,EffectivePlies,Symmetric,Balanced,AnglesFunction,LoadCase,Train,Test
Value,705,300,0.001575,10,8,2,True,True,harmlin,torsion,512,128


Now we have to recover the boundaries used during the design of experiments in order to be able to define the deisign space in which search for an optimum configuration. The dimension of the space in which each particle lives is given by the number of variables in each ply and the number of free plies

In [7]:
boundaries_df = pd.read_csv(input_folder + 'features_min_max.csv', sep=',')
boundaries_df.index = ['min', 'max']
boundaries = boundaries_df.to_numpy()
#boundaries = np.tile(boundaries_df.to_numpy(), 2)
interval = boundaries[None, 1, :] - boundaries[None, 0, :]
min_boundaries = boundaries[None, 0, :]
max_boundaries = boundaries[None, 1, :]

In [8]:
x_bounds_df = pd.read_csv(input_folder + 'X_bounds.csv', sep=',', index_col=0)
y_bounds_df = pd.read_csv(input_folder + 'Y_bounds.csv', sep=',', index_col=0)
x_bounds = x_bounds_df.to_numpy()
y_bounds = y_bounds_df.to_numpy()

## Import the trained network

First we have to import the network architecture and instanciate it:

In [9]:
D_in, H, D_out = 8, 64, 2
p = 0.2

class FFNN(torch.nn.Module):
    """ Implementation of FeedForward Neural Network """
    def __init__(self, D_in, H, D_out, p):
        super(FFNN, self).__init__()
        # First hidden layer
        self.W_1 = Parameter(init.xavier_normal_(torch.Tensor(H, D_in)))
        self.b_1 = Parameter(init.constant_(torch.Tensor(H), 0))
        # Second hidden layer
        self.W_2 = Parameter(init.xavier_normal_(torch.Tensor(H, H)))
        self.b_2 = Parameter(init.constant_(torch.Tensor(H), 0))
        # Third hidden layer
        #self.W_3 = Parameter(init.xavier_normal_(torch.Tensor(H, H)))
        #self.b_3 = Parameter(init.constant_(torch.Tensor(H), 0))
        # Output layer
        self.W_4 = Parameter(init.xavier_normal_(torch.Tensor(D_out, H)))
        self.b_4 = Parameter(init.constant_(torch.Tensor(D_out), 0))
        
        # define activation function in constructor
        self.activation_1 = torch.nn.ReLU()
        self.activation_2 = torch.nn.ReLU()
        #self.activation_3 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=p)

    def forward(self, x):
        x = F.linear(x, self.W_1, self.b_1)
        x = self.activation_1(x)
        x = self.dropout(x)
        x = F.linear(x, self.W_2, self.b_2)
        x = self.activation_2(x)
        x = self.dropout(x)
        #x = F.linear(x, self.W_3, self.b_3)
        #x = self.activation_3(x)
        #x = self.dropout(x)
        pred = F.linear(x, self.W_4, self.b_4)
        return pred
    
model = FFNN(D_in, H, D_out, p)

In [10]:
model.load_state_dict(torch.load(input_folder + 'weights_NN'))

<All keys matched successfully>

We can check if the weights are correct.

In [11]:
for name, param in model.named_parameters():
    print(name, param)

W_1 Parameter containing:
tensor([[-1.0844e-01, -4.9057e-01, -4.6628e-01,  6.9210e-01, -2.3332e-01,
          4.2882e-01, -4.1004e-01, -1.8674e-01],
        [-1.4891e-01, -3.9336e-01, -9.4734e-02,  5.2422e-02,  1.7628e-01,
          2.3287e-01, -8.9333e-03,  3.8331e-01],
        [-1.8496e-01,  2.4584e-02,  2.1688e-02,  3.7966e-02,  5.9815e-02,
         -5.8522e-01, -5.1259e-01,  1.1237e-01],
        [ 1.1920e-03, -5.3885e-03,  5.2022e-03, -2.2404e-03,  2.4978e-02,
         -2.0544e-02,  2.3576e-02, -6.3236e-01],
        [ 7.2022e-02, -5.1027e-02, -2.5911e-02,  5.7195e-01, -5.0238e-02,
          6.3549e-03,  6.9255e-03, -9.6424e-03],
        [-8.7163e-02,  1.0295e-01, -1.1280e-01, -2.4809e-01,  6.9280e-02,
          3.9446e-01, -5.7644e-01, -1.3134e-01],
        [-5.4915e-01,  1.8107e-01,  3.1066e-01, -4.0122e-02,  3.0949e-02,
         -7.8460e-02,  1.0198e-02, -1.1879e-01],
        [ 1.3140e-02,  5.5321e-03,  1.2650e-02,  7.3111e-01, -3.6382e-02,
         -2.8615e-02, -1.0237e-02, -2.4

       requires_grad=True)
W_2 Parameter containing:
tensor([[-1.6548e-01, -6.6943e-03, -4.8915e-03,  ..., -2.8771e-02,
         -6.9771e-05, -1.5098e-01],
        [-3.8894e-03, -1.0169e-04, -1.8631e-01,  ...,  2.3320e-02,
          1.1817e-04, -5.7920e-03],
        [-2.0205e-01,  1.6208e-02,  6.2865e-04,  ..., -4.2851e-02,
         -1.5836e-04, -2.9667e-06],
        ...,
        [-2.6094e-05,  5.4215e-02,  3.3715e-05,  ..., -6.0024e-02,
          8.5578e-05,  7.4026e-02],
        [-2.0628e-01,  5.4677e-03,  4.8458e-03,  ..., -1.0183e-02,
          1.4416e-04, -5.7903e-04],
        [-1.5925e-01,  3.8437e-04, -5.5289e-04,  ..., -2.9123e-03,
         -8.7245e-06, -2.7485e-02]], requires_grad=True)
b_2 Parameter containing:
tensor([ 2.2950e-02, -2.6891e-02,  4.0276e-02,  4.8653e-02,  4.3052e-02,
        -2.9746e-01, -2.7227e-01,  4.2470e-02,  3.6495e-02,  3.5921e-02,
         5.1349e-02, -9.1147e-02,  3.7366e-02,  5.4128e-02,  3.2487e-02,
         1.1712e-02,  7.1657e-02,  5.3352e-02,  5.

## **Particle Swarm Optimizer**

We start defining some functions.


In [12]:
def a_max(height, mesh_size, amplitude, phase_shift, omega, beta, k_max):
    """ Function that calculate the new amplitude of the fiber path such that the
        manifactuing constraint is not violated """
    x = np.linspace(mesh_size / 2, h - mesh_size / 2, 1000) 
    dy = omega * (2 * np.pi / h) * amplitude * np.cos(omega * (2 * np.pi / h) * x + phase_shift) + np.tan(beta )
    ddy = - (omega**2) * (4 * (np.pi**2) / (h**2)) * amplitude * np.sin(omega * (2 * np.pi / h) * x + phase_shift)
    k = ddy / ((1 + dy**2)**(1.5))

    max_value = np.amax(abs(k))
    max_pos = np.where(abs(k) == max_value)

    if max_value > k_max:
        def curvature(a_hat):
            dy = omega * (2 * np.pi / h) * a_hat * np.cos(omega * (2 * np.pi / h) * x[max_pos[0][0]] + phase_shift) + np.tan(beta)
            ddy = - (omega**2) * (4 * (np.pi**2) / (h**2)) * a_hat * np.sin(omega * (2 * np.pi / h) * x[max_pos[0][0]] + phase_shift)
            return k_max - abs(ddy / ((1 + dy**2)**(1.5)))
        new_a = fsolve(curvature, amplitude)
    else:
        new_a = amplitude
    
    return new_a

Let's define the function to initialize the particles position and velocity. In this initialization, and in all the other steps, we have to check the manufacturing constraints of the generated candidate paths.

In [13]:
def particles_init(particles, interval, min_boundaries, eff_plies):
    """ Initialization function for the velocity and position of the swarm """
    position = None
    velocity = None
    deg2rad = np.pi / 180
    for ply in range(1, eff_plies + 1):
        design = pyDOE.lhs(interval.shape[1], particles)
        x = design * interval + min_boundaries
        for j in range(0, particles):
            new_a = a_max(h, mesh_size, x[j, 0], x[j, 1] * deg2rad, x[j, 2], x[j, 3] * deg2rad, k_max)
            x[j, 0] = new_a
        
        if position is None:
            position = np.copy(x)
        else:
            position = np.column_stack((position, x))
            
    for ply in range(1, eff_plies + 1):
        design = pyDOE.lhs(interval.shape[1], particles)
        x = design * interval + min_boundaries
        for j in range(0, particles):
            new_a = a_max(h, mesh_size, x[j, 0], x[j, 1] * deg2rad, x[j, 2], x[j, 3] * deg2rad, k_max)
            x[j, 0] = new_a

        if velocity is None:
            velocity = np.copy(x) * 0.1
        else:
            velocity = np.column_stack((velocity, x)) * 0.4
    return position, velocity

Let's define the function to **let the particles flyyyyy**.

In [59]:
def fly(param, pos, vel, cognitive_pos_best, social_pos_best, boundaries, max_iter, run):
    """ Function to let the swarm flies in the design space"""
    deg2rad = np.pi / 180
    w_max, w_min, c1, c2 = param
    w = w_max - (w_max - w_min) / max_iter * run
    
    inertial_velocity = w * vel
    cognitive_velocity = c1  * np.random.rand(particles, dimensions) * (cognitive_pos_best - pos)
    social_velocity = c2 * np.random.rand(particles, dimensions) * (np.tile(social_pos_best, (particles, 1)) - pos)
    
    # Update velocities
    new_vel = inertial_velocity + cognitive_velocity + social_velocity
    
    # Update positions
    new_pos = pos + new_vel
    
    # Correct particles position
    for ply in range(0, eff_plies):
        # Generate a new temporary array for each ply
        _new_pos = copy(new_pos[:, ply * interval.shape[1]: ply * interval.shape[1] + interval.shape[1]])
        _new_vel = copy(new_vel[:, ply * interval.shape[1]: ply * interval.shape[1] + interval.shape[1]])
        
        # Generate an array of min and max values of the features
        min_array = np.tile(boundaries[0, :], (particles, 1))
        max_array = np.tile(boundaries[1, :], (particles, 1))
        
        # Correct out-of-boundaries features position
        #print("Init : {:+04f} {:+04f} {:+04f} {:+04f}".format(_new_vel[0, 0], _new_vel[0, 1], _new_vel[0, 2], _new_vel[0, 3]))

        _new_vel[_new_pos < min_array] = -0.1 * _new_vel[_new_pos < min_array]
        _new_vel[_new_pos > max_array] = -0.1 * _new_vel[_new_pos > max_array]
        
        #print("Init : {:+04f} {:+04f} {:+04f} {:+04f}".format(_new_vel[0, 0], _new_vel[0, 1], _new_vel[0, 2], _new_vel[0, 3]))
        _new_pos[_new_pos < min_array] = min_array[_new_pos < min_array]
        _new_pos[_new_pos > max_array] = max_array[_new_pos > max_array]
        
        _new_pos[_new_pos < min_array] = _new_pos[_new_pos < min_array] + _new_vel[_new_pos < min_array]
        _new_pos[_new_pos > max_array] = _new_pos[_new_pos > max_array] + _new_vel[_new_pos > max_array]
        
        # Correct manufacturing constraints
        for j in range(0, particles):
            new_a = a_max(h, mesh_size, _new_pos[j, 0], _new_pos[j, 1] * deg2rad,
                          _new_pos[j, 2], _new_pos[j, 3] * deg2rad, k_max)
            
            if new_a != _new_pos[j, 0]:
                #print('dentro')
                #print(_new_pos[j, 0])
                _new_pos[j, 0] = new_a
                #print(_new_pos[j, 0])
        
        new_pos[:, ply * interval.shape[1]: ply * interval.shape[1] + interval.shape[1]] = _new_pos
        new_vel[:, ply * interval.shape[1]: ply * interval.shape[1] + interval.shape[1]] = _new_vel
                
    return new_pos, new_vel

We need to define the function that, given the particles positions, evaluates the buckling load and the pre-bluckling stiffness.

In [60]:
def net_prediction(model, position, x_bounds, y_bounds):
    """ Function to predict the scores through a neural network"""
    model.eval()
    _X, _, _ = range_norm(position, x_min=x_bounds[0, :], x_max=x_bounds[1, :])
    X = Variable(torch.from_numpy(_X.astype('float32')))
    _Y = model(X)
    Y = reverse_range_norm(_Y.detach().numpy(), y_bounds[0, :], y_bounds[1, :])
    return Y[:, 0], Y[:, 1]

Now we need to define some parameters of the optimizer:

In [61]:
particles = 80
tot_opt = 2
dimensions = interval.shape[1] * eff_plies
max_iter = 1000
w_max = 1.4
w_min = 0.3
c1 = 2
c2 = 2
param = [w_max, w_min, c1, c2]

In [62]:
tic = time.time()

max_buckling = []
opt_config = []
    
for opt in range(tot_opt):
    
    print("Optimization: {:02d}".format(opt+1))
    
    # Initialize particles positions and velocities
    pos, vel = particles_init(particles, interval, min_boundaries, eff_plies)
    #print("Init : {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f}".format(pos[0, 0], pos[0, 1], pos[0, 2], pos[0, 3], pos[0, 4], pos[0, 5], pos[0, 6], pos[0, 7]))

    # Predict the score associated with each particle
    bck, stiff = net_prediction(model, pos, x_bounds, y_bounds)

    # The score is the inverse of the buckling load
    score = 1 / bck

    # Set to zero score value less than zero. We look for the highest positive buckling value.
    score[score < 0] = np.inf

    # initialization of particles and swarm scores
    cognitive_best = copy(score)
    cognitive_pos_best = copy(pos)
    social_best = score[np.argmin(score)]
    social_pos_best = pos[np.argmin(score), :]
    buckling_hist = []  # contains envelope of best buckling

    for run in range(max_iter):
        # Fly
        #print("Pre fly : {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f}".format(pos[0, 0], pos[0, 1], pos[0, 2], pos[0, 3], pos[0, 4], pos[0, 5], pos[0, 6], pos[0, 7]))
        pos, vel = fly(param, pos, vel, cognitive_pos_best, social_pos_best, boundaries, max_iter, run)
        #print("Post fly: {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f} {:+04f}".format(pos[0, 0], pos[0, 1], pos[0, 2], pos[0, 3], pos[0, 4], pos[0, 5], pos[0, 6], pos[0, 7]))

        # Evaluate score
        bck, stiff = net_prediction(model, pos, x_bounds, y_bounds)
        score = 1 / bck
        score[score < 0] = np.inf

        cognitive_best[cognitive_best > score] =  score[cognitive_best > score]
        cognitive_pos_best[cognitive_best > score, :] =  pos[cognitive_best > score, :]

        if score[np.argmin(score)] < social_best:
            social_best = score[np.argmin(score)]
            social_pos_best = pos[np.argmin(score), :]
            #print("Iteration: {:04d}, actual maximum torsional buckling: {}".format(run, bck[np.argmin(score)]))
            
    max_buckling.append(1 / social_best)
    opt_config.append(social_pos_best)

Optimization: 01
Optimization: 02


In [63]:
opt_config = np.array(opt_config)
max_buckling = np.array(max_buckling)
max_buckling = max_buckling.reshape((-1, 1))
results = np.hstack((max_buckling, opt_config))
columns = ['Buckling',
           'Amplitude1', 'PhaseShift1', 'Omega1', 'Beta1',
           'Amplitude2', 'PhaseShift2', 'Omega2', 'Beta2']
results_df = pd.DataFrame(results, columns=columns)
results_df

,Buckling,Amplitude1,PhaseShift1,Omega1,Beta1,Amplitude2,PhaseShift2,Omega2,Beta2
0,46610.092183,200.000000,90.000000,0.300813,90.0,5.081572,-90.0,2.0,7.015208
1,47255.633413,-114.293289,-21.889275,0.000000,90.0,200.000000,90.0,2.0,-90.000000


In [19]:
output_folder = './' + load_case + '/' + stacking_sequence + '/' + data_set + '/' + fiber_path + '/'
try:
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
except OSError:
    print('Error: Creating directory. ' + output_folder)
results_df.to_csv(output_folder + 'results.csv', index=False, float_format='%.4f')